# Unsupervised learning - Clustering
## Agglomerative clustering

Follow:
- _Introduction to Machine Learning_ [Chapter 3](https://github.com/amueller/introduction_to_ml_with_python/blob/master/03-unsupervised-learning.ipynb) **Section 3.5.2 Agglomerative Clustering** (p.184-188)






In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import mglearn

## Introduction to Agglomerative clustering
code from Introduction to Machine Learning with Python Chapter 3.5.2:[Agglomerative Clustering](https://github.com/amueller/introduction_to_ml_with_python/blob/master/03-unsupervised-learning.ipynb)

In [ ]:
mglearn.plots.plot_agglomerative_algorithm()

>Initially, each point is its own cluster. Then, in each step, the two clusters that are closest are merged. In the first four steps, two single-point clusters are picked and these are joined into two-point clusters. In step 5, one of the two-point clusters is extended to a third point, and so on. In step 9, there are only three clusters remaining. As we specified that we are looking for three clusters, the algorithm then stops.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=1)

agg = AgglomerativeClustering(n_clusters=3)
assignment = agg.fit_predict(X)

mglearn.discrete_scatter(X[:, 0], X[:, 1], assignment)
plt.legend(["Cluster 0", "Cluster 1", "Cluster 2"], loc="best")
plt.xlabel("Feature 0")
plt.ylabel("Feature 1");

### How clusters are merged
We need to define:
1. the *distance metric* and 
2. the *linkage type*

The default *distance metric* is *euclidean* distance

The following four *linkage types* are implemented in scikit-learn:

**ward**  
The default choice, ward picks the two clusters to merge such that the variance in the distance metric within all clusters increases the least. This often leads to clusters that are relatively equally sized

**average**  
average linkage merges the two clusters that have the smallest average distance between all their points

**complete**  
complete linkage (also known as maximum linkage) merges the two clusters that have the smallest maximum distance between their points

**single**  
merges the two clusters that have the smallest minimum distance between their points

## Dendrogram to visualize cluster formation

In [ ]:
X, y = make_blobs(random_state=0, n_samples=12)

agg = AgglomerativeClustering(n_clusters=3)
assignment = agg.fit_predict(X)

mglearn.discrete_scatter(X[:, 0], X[:, 1], assignment)
plt.legend(["Cluster 0", "Cluster 1", "Cluster 2"], loc="best")
plt.xlabel("Feature 0")
plt.ylabel("Feature 1");

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage


# Apply the linkage function to the data array X
# returns an array that specifies the distances
# bridged when performing agglomerative clustering

linkage_array = linkage(X, metric='euclidean', method='ward')

# Now we plot the dendrogram for the linkage_array containing the distances
# between clusters
dendrogram(linkage_array)

# mark the cuts in the tree that signify two or three clusters
ax = plt.gca()
bounds = ax.get_xbound()
ax.plot(bounds, [7.25, 7.25], '--', c='k')
ax.plot(bounds, [4, 4], '--', c='k')

ax.text(bounds[1], 7.25, ' two clusters', va='center', fontdict={'size': 15})
ax.text(bounds[1], 4, ' three clusters', va='center', fontdict={'size': 15})
plt.xlabel("Sample index")
plt.ylabel("Cluster distance");

### Verifying our small example

In [ ]:
#TODO: Create array, call linkage(), call dendogram()

In [ ]:
Xs = np.array([[0.5, 0.5], [0.5, 1], [1.5, 1.5]])
Xs

In [ ]:
# cluster_id1, cluster_id2, distance, num_of_elements in resulting cluster
la = linkage(Xs, metric='euclidean', method='complete')
la

In [ ]:
dendrogram(la)

### Visualizing cluster centres
Agglomerative clustering does not provide an attribute with cluster centers. We can calculate this ourselves

In [ ]:
agg.labels_

In [ ]:
df = pd.DataFrame(X, columns=['f1', 'f2'])
df['label'] = agg.labels_
df['label'] = df['label'].astype('category') #seaborn plots will be nicer
df

In [ ]:
centers = df.groupby(by='label').mean()
centers

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
ax = sns.scatterplot(x='f1', y='f2', hue='label', ax=ax, data=df)

centers.plot.scatter(x='f1', y='f2', ax=ax, marker='x', s=80, color='black');

## Agglomerative clustering on a k-means failure case

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

X_varied, y_varied = make_blobs(n_samples=200,
                                cluster_std=[1.0, 2.5, 0.5],
                                random_state=170)
y_pred = KMeans(n_clusters=3, random_state=0).fit_predict(X_varied)

mglearn.discrete_scatter(X_varied[:, 0], X_varied[:, 1], y_pred)
plt.legend(["cluster 0", "cluster 1", "cluster 2"], loc='best')
plt.xlabel("Feature 0")
plt.ylabel("Feature 1")
plt.title("K-means");

In [ ]:
agg = AgglomerativeClustering(n_clusters=3)
assignment = agg.fit_predict(X_varied)

mglearn.discrete_scatter(X_varied[:, 0], X_varied[:, 1], assignment)
plt.legend(["Cluster 0", "Cluster 1", "Cluster 2"], loc="best")
plt.xlabel("Feature 0")
plt.ylabel("Feature 1")
plt.title("Agglomerative");

## Wine dataset

In [ ]:
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
X, _ = load_wine(return_X_y=True, as_frame=True)
X = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)
X.info()

### Dendrogram to cluster the wines

In [ ]:
# Apply the ward clustering to the data array X
# The SciPy ward function returns an array that specifies the distances
# bridged when performing agglomerative clustering
linkage_array = linkage(X, method='complete')
# Now we plot the dendrogram for the linkage_array containing the distances
# between clusters
dendrogram(linkage_array);

In this representation, we see that there is not much distance (y-axis value) between choosing 3 or 4 clusters.

### Dendogram to cluster the features 

In [ ]:
# Apply the ward clustering to the data array X
# The SciPy ward function returns an array that specifies the distances
# bridged when performing agglomerative clustering
linkage_array = linkage(X.transpose(), method='complete')
# Now we plot the dendrogram for the linkage_array containing the distances
# between clusters
dendrogram(linkage_array, labels=list(X.transpose().index.values), orientation='left');
plt.xticks(rotation=90);

### Dendogram to cluster features based on correlation
Instead of euclidean distance, spearman correlation is used to calculate *distance*

More precisely, distance = 1 - correlation.

In [ ]:
# Reference: https://github.com/fastai/fastbook/blob/master/utils.py

from scipy.cluster import hierarchy as hc
from scipy.stats import spearmanr
import matplotlib.pyplot as plt

def cluster_columns(df, figsize=(10,6), font_size=12):
    corr = np.round(spearmanr(df).correlation, 4) #correlation matrix
    # 1-corr is the distance between columns in rank-correlation space.
    corr_condensed = hc.distance.squareform(1-corr) # create a condensed distance matrix
    # we could probably run the linkage on the correlation matrix directly?
    # corr_condensed = corr
    z = hc.linkage(corr_condensed, method='average') # calculate hierachical clustering on condensed distance matrix
    fig = plt.figure(figsize=figsize)
    hc.dendrogram(z, labels=df.columns, orientation='left', leaf_font_size=font_size)
    plt.show()

In [ ]:
cluster_columns(X)

### Agglomerative clustering with PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import calinski_harabasz_score

agg = AgglomerativeClustering(n_clusters=3)
assignment = agg.fit_predict(X)

pca = PCA(n_components=2)
pca.fit(X)
X_2D = pca.transform(X)

mglearn.discrete_scatter(X_2D[:, 0], X_2D[:, 1], assignment)
plt.legend(["Cluster 0", "Cluster 1", "Cluster 2", "Cluster 3"], loc="best")
plt.xlabel("PC 0")
plt.ylabel("PC 1")
plt.title("calinski-harabasz score {:.1f}".format(calinski_harabasz_score(X, assignment)));

In [ ]:
agg = AgglomerativeClustering(n_clusters=4)
assignment = agg.fit_predict(X)

pca = PCA(n_components=2)
pca.fit(X)
X_2D = pca.transform(X)

mglearn.discrete_scatter(X_2D[:, 0], X_2D[:, 1], assignment)
plt.legend(["Cluster 0", "Cluster 1", "Cluster 2", "Cluster 3"], loc="best")
plt.xlabel("PC 0")
plt.ylabel("PC 1")
plt.title("calinski-harabasz score {:.1f}".format(calinski_harabasz_score(X, assignment)));